In [2]:
!pip install transformers


   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------  10.0/10.1 MB 61.8 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 48.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 45.3 MB/s eta 0:00:00


In [30]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import time
import pandas as pd
import kagglehub
from typing import List
import string

path = kagglehub.dataset_download("trainingdatapro/ocr-receipts-text-detection")

In [15]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

c:\Users\gta76\Desktop\project\ml-project\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gta76\.cache\huggingface\hub\models--microsoft--trocr-base-printed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritte

In [56]:
valid_data = [
    ['BANANAS', 'FRAP'],
    ['R-CARROTS SHREDDED 10 OZ', 'R-CUCUMBERS PERSIAN 1 LB', 'TOMATOES CRUSHED NO SALT', 'TOMATOES WHOLE NO SALT W/BASIL', 
     'ORGANIC OLD FASHIONED OATMEAL', 'MINI-PEARL TOMATOES..', 'PKG SHREDDED MOZZARELLA LITE T', 'EGGS 1 DOZ ORGANIC BROWN.', 
     'BEANS GARBANZO', 'SPROUTED CA STYLE', 'A-AVOCADOS HASS BAG 4CT', 'A-APPLE BAG JAZZ 2 LB', 'A-PEPPER BELL EACH XL RED', 
     'GROCERY NON TAXABLE', 'BANANAS ORGANIC', 'CREAMY SALTED PEANUT BUTTER', 'WHL WHT PITA BREAD', 'GROCERY NON TAXABLE'],
    ['GV OATMEAL', 'OT 200Z TUM', 'M ATHLETICS', 'DEXAS 15X20'],
    ['TATER TOTS', 'HARD/PROV/DC', 'SNACK BARS', 'HRI CL CHS', 'HRI CL CHS', 'HRI CL CHS', 'HRI 12 U SG', 'HRI CL PEP', 'EARBUDS',
      'SC BCN CHDDR', 'ABF THINBRST', 'ABF THINBRST', 'HARD/PROV/DC', 'DV RSE OIL M', 'APPLE 3 BAG', 'STOK LT SWT', 'PEANUT BUTTER',
      'AVO VERDE', 'ROLLS', 'BTS DRY BLON', 'GALE', 'TR HS FRM 4', 'BAGELS', 'GV SLIDERS', 'ACCESSORY', 'CHEEZE IT', 'RITZ', 'RUFFLES', 'GV HNY GRMS'],
    ['SW HRO FGHTR'],
    ['KTTL SEA SALT POT CHP', 'BRAIDED BRIOCHE', 'CHEF PLATE MEAL'],
    ['Woman', 'Ham Cheese', 'Ice Java Tea', 'Mineral Water', 'Black & White'],
    ['6 WING PLATE', 'ASST 27', 'CUTIE CAR'],
    ['FF BS BREAST', 'KS DICED TOM', 'JACKORGSALSE', '18CT EGGS', 'GRAPE TOMATO', 'ECO HALF PAN', 'GRND TURKEY', 'CHPD ONION', 'MONT JACK 2#'],
    ['N/YORK TX GRLC', 'CHICKEN WINGS', 'PICSWEET BLEND', 'PICSWEET GREENS', 'CHICKEN WINGS', 'DIET LN GRND B', 'PORK LOIN CHOP', 'DRUMSTICKS',
     'EYE RND STK FP', 'EYE RND STK FP', 'BEEF FOR STEW', 'PORK 1/2 LOIN', 'B/BEST S STEAK', 'BAR S MEAT FRNK', '0/M HAM', 'WINCO NOODLES', 'TNDRBRD CHIX BR'
     'KRFT DELUXE MAC', 'EGGO WAFFLE', 'L/D NUTTY BARS', 'L\D OATMEAL CRM', 'LINKS MILD', 'KEEB TOWNHOUSE', 'TURKEY FRANK', 'RESERS POT SLD', 'DM SPAG SAUCE',
     'HUNTS MANWICH', 'DM PNAPL CHNKY', 'SUNBEAN BUNS', 'WINCO SNDWCH WT', 'WINCO HOMO MILK', 'EGGS'],
    ['LAZENBY WORCESTER SAUCE', 'MILKY BAR CHOC', 'SMOKED VIENNAS', 'PEALED PEACHES', 'MEDITERRANEAN MIX', 'SPAR COOKING OIL',
     'F/L ENGLISH CUCUMB', 'NESTLE AERO', 'CADBURY DAIRY MI', 'GRAPES MIXED TUB', 'TASTIC RICE', 'BLACK CAT SMOOTH', 'CARRIER BAG 24L', 'BANANAS LOOSE'],
    ["PL TORTILLA'S", 'CAGE FREE ALL WHIT', 'BLACK BEANS', 'Frozen Mangoes 16o', 'Whole Strawberries', 'OG LF COTTAGE CHEE', 'MAHI MAHI FILLETS', '$2 off (1) WC Fill',
     'California Harvest', 'PLUMS BLACK CV'],
    ['SW FIGURES', 'TOOTHBRUSH', 'WOMEN SLIPPE'],
    ['GIFT CARD'],
    ['EQUATE LINER', 'VFUS ENG POM', 'DAWN ORIG', '50YD PKGTAPE', '50YD PKGTAPE', 'LINT POLLER2'],
    ['DIABETES'],
    ['6 WING PLATE', 'ASST 27', 'CUTIE CAR'],
    ['BANANAS', 'BEVERAGE', 'OS CRAN POM', 'STRWBRY CC', 'CAMPARI TOM', 'KFT SINGLES', 'HARD SALAMI', 'AVOCADO', 'PILLS WHITE', 'SH NYLON COL', 'HAND CLEANER', 'INJECTR CLNR'],
    ['GV WATER', 'GV WATER', 'GREAT VALUE', 'GREAT VALUE', 'GAIN LFE', 'GAIN', 'GV VEG OIL', 'SNICKER FS', 'GDBR FS 6PK', 'MIXED CHOC', 'CHERRIES', 'GAIN', 'HY PSTLEG PM',
     'BATTERIES', 'GV 20Z MINI', 'CRNBRY SAUCE', 'SPREADS', 'EGGS 6CT'],
    ['GRILL COVER', 'FIBER CHOICE', 'CELERY HEART', 'RED GRAPE'],
]

In [21]:
def extract_text_trocr_lines(image_path):
    image = Image.open(image_path).convert("RGB")
    width, height = image.size

    # Разбиваем изображение на строки
    line_height = 40  # Высота строки (можно подобрать)
    texts = []

    for y in range(0, height, line_height):
        cropped_line = image.crop((0, y, width, min(y + line_height, height)))
        pixel_values = processor(images=cropped_line, return_tensors="pt").pixel_values
        
        generated_ids = model.generate(pixel_values)
        line_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        if line_text.strip():
            texts.append(line_text)

    return " ".join(texts)

In [23]:
results = []
for i in range(int(len(valid_data) / 2)):
    image_path = f'{path}/images/{i}.jpg'

    start_time = time.time()
    recognized_text = extract_text_trocr_lines(image_path)
    end_time = time.time()
    processing_time = end_time - start_time
    
    results.append({
        "image_path": image_path,
        "ground_truth": valid_data[i],
        "recognized_text": recognized_text,
        "processing_time": processing_time,
    })

results_df = pd.DataFrame(results)
results_df

,image_path,ground_truth,recognized_text,processing_time
0,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[BANANAS, FRAP]",- CASHIER CASHIER - ........ WALMART ALWAYS LO...,45.765995
1,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[R-CARROTS SHREDDED 10 OZ, R-CUCUMBERS PERSIAN...",THANK YOU FOR FULL WWW. *** TRADER JOE'S AMOUN...,72.264516
2,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[GV OATMEAL, OT 200Z TUM, M ATHLETICS, DEXAS 1...","AMOUNT THANK YOU! ID #: 788504072,WALMART.COM ...",52.375142
3,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[TATER TOTS, HARD/PROV/DC, SNACK BARS, HRI CL ...",AMOUNT AMOUNT AMOUNT AMOUNT NO WIN ST000 TEEN ...,64.085966
4,C:\Users\gta76\.cache\kagglehub\datasets\train...,[SW HRO FGHTR],WALMARKU SAVE MONEY.LIVEBETTER. 1 (8) PLEASE S...,37.956999
5,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[KTTL SEA SALT POT CHP, BRAIDED BRIOCHE, CHEF ...",WWW. CASHIER NO: 5:00 PM WWW.LA.COM SR007 AD |...,321.784266
6,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[Woman, Ham Cheese, Ice Java Tea, Mineral Wate...",AMOUNT ITEMS MOMLATOVE *** CREPERIE CASHIER AM...,97.640744
7,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[6 WING PLATE, ASST 27, CUTIE CAR]",1 1 WWWWWWWWWWWWWWWW. PLEASE TO WIN $1000 RECE...,99.018761
8,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[FF BS BREAST, KS DICED TOM, JACKORGSALSE, 18C...",WW.SALBOVE. SR: COSTCO - 16375 N. WASHINSTON S...,63.131017
9,C:\Users\gta76\.cache\kagglehub\datasets\train...,"[N/YORK TX GRLC, CHICKEN WINGS, PICSWEET BLEND...",THANK YOU FOR A FREE - - AM WINCO SR: *** *** ...,83.370610


In [58]:
def normalize_text(text: str) -> str:
    return text.lower().translate(str.maketrans('', '', string.punctuation))

def calculate_accuracy(recognized: List[str], valid: List[str]) -> dict:
    recognized_set = {normalize_text(word) for word in recognized}
    valid_set = {normalize_text(word) for word in valid}
    
    true_positives = len(recognized_set & valid_set)
    false_positives = len(recognized_set - valid_set)
    false_negatives = len(valid_set - recognized_set)
    
    precision = true_positives / (true_positives + false_positives) if recognized_set else 0
    recall = true_positives / (true_positives + false_negatives) if valid_set else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0
    
    return {
        "precision": precision * 100,
        "recall": recall * 100,
        "f1_score": f1_score * 100,
        "accuracy": (true_positives / len(valid)) * 100 if valid else 0
    }

for i in range(int(len(valid_data) / 2)):
    print(calculate_accuracy(results_df['recognized_text'][i].split(), valid_data[i]))

{'precision': 1.5151515151515151, 'recall': 50.0, 'f1_score': 2.9411764705882355, 'accuracy': 50.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
{'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'accuracy': 0.0}
